<a href="https://colab.research.google.com/github/shreya2923/Normal-dataset/blob/main/Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
!pip list


Package                                  Version
---------------------------------------- --------------------
absl-py                                  1.4.0
accelerate                               1.12.0
access                                   1.1.10.post3
affine                                   2.4.0
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
aiosqlite                                0.22.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                                   5.5.0
annotated-doc                            0.0.4
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                        

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
from google.colab import files
uploaded = files.upload()


Saving Normal_S21_L.csv to Normal_S21_L.csv


In [6]:
# Basic Neural Network on your CSV dataset (Normal_S21_L.csv)

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

# =========================
# Load Dataset
# =========================
# Make sure to upload 'Normal_S21_L.csv' to the Colab environment (e.g., to /content/).
# You can do this by clicking the folder icon on the left sidebar, then 'Upload to session storage'.
csv_path = "/content/Normal_S21_L.csv"
df = pd.read_csv(csv_path)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# =========================
# Split features & target
# =========================
# Assumes last column is label
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# In this case, 'S21_Phase(rad)' is a continuous value, indicating a regression task.
# The LabelEncoder step below is for classification and is skipped because y is numeric.
# For regression, we treat y as a continuous target.

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
# For regression, targets should be float32 and typically have shape (N, 1)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test  = torch.tensor(y_test,  dtype=torch.float32).unsqueeze(1)

input_size = X_train.shape[1]
# For regression, num_classes is not applicable in the same way as classification.
# The output layer will have 1 feature for the continuous prediction.
output_features = 1

print("Input features:", input_size)
print("Output features (for regression):", output_features)

# =========================
# Define Neural Network
# =========================
class NormalNet(nn.Module):
    def __init__(self, in_features, h1=16, h2=8, out_features=1): # out_features changed to 1 for regression
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x) # No activation for the output layer in basic regression
        return x

model = NormalNet(
    in_features=input_size,
    h1=32,
    h2=16,
    out_features=output_features # Use output_features for model definition
)

# =========================
# Training Setup
# =========================
criterion = nn.MSELoss() # Changed to MSELoss for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50

for epoch in range(epochs):
    model.train() # Set model to training mode
    optimizer.zero_grad()

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}  Loss: {loss.item():.4f}")

# =========================
# Evaluate on Test Data
# =========================
model.eval() # Set model to evaluation mode
with torch.no_grad():
    test_predictions = model(X_test)
    # For regression, we evaluate using a metric like Mean Squared Error
    test_loss = criterion(test_predictions, y_test)

    # Convert tensors to numpy for sklearn metrics if desired
    # y_test_np = y_test.numpy()
    # test_predictions_np = test_predictions.numpy()
    # mse = mean_squared_error(y_test_np, test_predictions_np)

print(f"\nTest MSE: {test_loss.item():.4f}")


Dataset shape: (1000, 3)
Columns: ['Frequency(Hz)', 'S21_Magnitude', 'S21_Phase(rad)']
Input features: 2
Output features (for regression): 1
Epoch 10/50  Loss: 5.5625
Epoch 20/50  Loss: 5.2776
Epoch 30/50  Loss: 4.9528
Epoch 40/50  Loss: 4.5785
Epoch 50/50  Loss: 4.1446

Test MSE: 4.1240


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50

for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}  Loss = {loss.item():.4f}")


# Evaluate
with torch.no_grad():
    logits = model(X_test)
    preds = torch.argmax(logits, dim=1)
    accuracy = (preds == y_test).float().mean()

print("\nTest Accuracy:", float(accuracy))


Epoch 10/50  Loss = -0.0000
Epoch 20/50  Loss = -0.0000
Epoch 30/50  Loss = -0.0000
Epoch 40/50  Loss = -0.0000
Epoch 50/50  Loss = -0.0000

Test Accuracy: 0.0


In [20]:
import pandas as pd

file_name = list(uploaded.keys())[0]
print("Loaded file:", file_name)

df = pd.read_csv(file_name)
df.head()


Loaded file: Normal_S21_L.csv


,Frequency(Hz),S21_Magnitude,S21_Phase(rad)
0,1.000000e+09,0.000002,2.317889
1,1.001811e+09,0.000002,2.318019
2,1.003622e+09,0.000002,2.318149
3,1.005433e+09,0.000002,2.318278
4,1.007243e+09,0.000002,2.318408


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 👉 Paste your GitHub RAW CSV link here
url = "import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 👉 Paste your GitHub RAW CSV link here
url = "https://github.com/shreya2923/Normal-dataset/blob/main/Normal_S21_L.csv"

# Load dataset
df = pd.read_csv(url)

print("Dataset loaded successfully!")
print("Shape:", df.shape)
df.head()


# Load dataset
df = pd.read_csv(url)

print("Dataset loaded successfully!")
print("Shape:", df.shape)
df.head()


In [8]:
url = "https://raw.githubusercontent.com/shreya2923/Normal-dataset/main/Normal_S21_L.csv"
df = pd.read_csv(url)
df.head()

,Frequency(Hz),S21_Magnitude,S21_Phase(rad)
0,1.000000e+09,0.000002,2.317889
1,1.001811e+09,0.000002,2.318019
2,1.003622e+09,0.000002,2.318149
3,1.005433e+09,0.000002,2.318278
4,1.007243e+09,0.000002,2.318408


In [11]:
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default

# Authenticate and create the gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# Define the name for your new Google Sheet
spreadsheet_name = 'Normal_S21_L_DataFrame_Export'

try:
    # Try to open an existing spreadsheet
    sh = gc.open(spreadsheet_name)
    print(f"Opened existing spreadsheet: {spreadsheet_name}")
except gspread.exceptions.SpreadsheetNotFound:
    # If not found, create a new one
    sh = gc.create(spreadsheet_name)
    print(f"Created new spreadsheet: {spreadsheet_name}")

worksheet = sh.get_worksheet(0)

set_with_dataframe(worksheet, df)

print(f"DataFrame successfully written to Google Sheet: {sh.url}")


Created new spreadsheet: Normal_S21_L_DataFrame_Export
DataFrame successfully written to Google Sheet: https://docs.google.com/spreadsheets/d/1UiUSqabw7L2K-d-AXqaLjHvLb4GRnQOIRgAHHq3VaNw


In [12]:
url = "https://raw.githubusercontent.com/shreya2923/Normal-dataset/main/Normal_S21_L.csv"
df = pd.read_csv(url)
df.head()

,Frequency(Hz),S21_Magnitude,S21_Phase(rad)
0,1.000000e+09,0.000002,2.317889
1,1.001811e+09,0.000002,2.318019
2,1.003622e+09,0.000002,2.318149
3,1.005433e+09,0.000002,2.318278
4,1.007243e+09,0.000002,2.318408


In [17]:
print("DataFrame Information:")
df.info()

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Frequency(Hz)   1000 non-null   float64
 1   S21_Magnitude   1000 non-null   float64
 2   S21_Phase(rad)  1000 non-null   float64
dtypes: float64(3)
memory usage: 23.6 KB


In [18]:
print("\nDataFrame Descriptive Statistics:")
display(df.describe())


DataFrame Descriptive Statistics:


,Frequency(Hz),S21_Magnitude,S21_Phase(rad)
count,1.000000e+03,1.000000e+03,1000.000000
mean,1.904523e+09,6.235451e-07,2.335856
std,5.230104e+08,3.897993e-07,0.008998
min,1.000000e+09,2.272805e-16,2.317889
25%,1.452261e+09,3.017266e-07,2.328166
50%,1.904523e+09,6.103106e-07,2.335892
75%,2.356784e+09,9.188945e-07,2.343618
max,2.809045e+09,2.248634e-06,2.351210
